In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
datasets_path = "../datasets_for_ml/"

reservas_futuras = pd.read_csv(datasets_path+'reservas_futuras.csv')
transacoes = pd.read_csv(datasets_path+'transacoes.csv')
cancelados = pd.read_csv(datasets_path+'cancelados.csv')
faltantes = pd.read_csv(datasets_path+'faltantes.csv')

transacoes['Date'] = pd.to_datetime(transacoes['Date'])
cancelados['Booking Date'] = pd.to_datetime(cancelados['Booking Date'])
cancelados['Cancel Date'] = pd.to_datetime(cancelados['Cancel Date'])
faltantes['Date'] = pd.to_datetime(faltantes['Date'])
reservas_futuras['Date'] = pd.to_datetime(reservas_futuras['Date'])

Vimos na análise dos dados que há interseções entre as tabelas de transações, cancelados e faltantes. Entretanto, suspeitamos que essas tabelas deveriam ser mutuamente exclusivas, de tal forma que não deveria haver interseções entre elas.

Vamos tomar a seguinte estratégia para lidar com esse problema:

* Todas as interseções de cancelados e faltantes com transações serão removidos, deixando as instâncias somente na tabela transações. Pois se o cliente está na tabela de transações significa que ele recebeu o serviço, portanto, não cancelou e nem faltou.

* Os clientes que estão na interseção de cancelados e faltantes serão removidos da tabela de faltantes e mantidos na tabela de cancelados. Se o cliente está na lista de cancelados, significa que ele cancelou e, portanto, não faltou.

Criando flags para auxiliar na remoção das instâncias

In [3]:
# Apenas para controle
transacoes.shape , cancelados.shape , faltantes.shape

((1862, 13), (240, 7), (59, 4))

In [4]:
cancelados['to_remove'] = cancelados.index
faltantes['to_remove'] = faltantes.index

In [5]:
remove = pd.merge(left = cancelados[['Booking Date','Code','to_remove']], 
               right = transacoes[['Date','Client']],
               left_on = ['Code','Booking Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True).drop_duplicates()['to_remove']


print(f"Quantidade de interseções entre cancelados e transacoes: {len(remove)}")

cancelados = cancelados.drop(remove)



Quantidade de interseções entre cancelados e transacoes: 56


In [6]:
remove = pd.merge(left = faltantes[['Date','Code','to_remove']], 
               right = transacoes[['Date','Client']],
               left_on = ['Code','Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True).drop_duplicates()['to_remove']


print(f"Quantidade de interseções entre faltantes e transacoes: {len(remove)}")
faltantes = faltantes.drop(remove)

Quantidade de interseções entre faltantes e transacoes: 1


In [7]:
remove = pd.merge(left = cancelados[['Booking Date','Code']], 
               right = faltantes[['Date','Code','to_remove']],
               left_on = ['Code','Booking Date'], 
               right_on=['Code','Date'],
               how='inner',
               indicator=True).drop_duplicates()['to_remove']


print(f"Quantidade de interseções entre cancelados e transacoes: {len(remove)}")
faltantes = faltantes.drop(remove)

Quantidade de interseções entre cancelados e transacoes: 4


In [8]:
cancelados = cancelados.drop('to_remove',axis=1)
faltantes = faltantes.drop('to_remove',axis=1)

In [9]:
# Apenas para controle
transacoes.shape , cancelados.shape , faltantes.shape

((1862, 13), (184, 7), (54, 4))

Nota-se que não há mais interseções entre as tabelas

In [10]:
print('Transações vs cancelados')
display(pd.merge(left = cancelados, 
               right = transacoes,
               left_on = ['Code','Booking Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True))

print("-"*100,'\nTransações vs faltantes')
display(pd.merge(left = faltantes, 
               right = transacoes,
               left_on = ['Code','Date'], 
               right_on=['Client','Date'],
               how='inner',
               indicator=True))

print("-"*100,'\nFaltantes vs cancelados')
display(pd.merge(left = cancelados, 
               right = faltantes,
               left_on = ['Code','Booking Date'], 
               right_on=['Code','Date'],
               how='inner',
               indicator=True))

Transações vs cancelados


,Cancel Date,Code,Service,Staff_x,Booking Date,Canceled By,Days,Receipt,Date,Description,...,Staff_y,Quantity,Amount,GST,PST,mes_nome,mes_n,dia_nome,dia_n,_merge


---------------------------------------------------------------------------------------------------- 
Transações vs faltantes


,Date,Code,Service,Staff_x,Receipt,Description,Client,Staff_y,Quantity,Amount,GST,PST,mes_nome,mes_n,dia_nome,dia_n,_merge


---------------------------------------------------------------------------------------------------- 
Faltantes vs cancelados


,Cancel Date,Code,Service_x,Staff_x,Booking Date,Canceled By,Days,Date,Service_y,Staff_y,_merge


In [11]:
faltantes = faltantes.rename({'Code':'Client'},axis=1)
cancelados = cancelados.rename({'Code':'Client','Booking Date': "Booking_Date"},axis=1)

# **Construção do dataset para treinamento, teste e validação**

O objetivo do projeto consiste em desenvolver um classificador que seja capaz de identificar os clientes que não sigam as políticas do salão. Portanto, as únicas informações que teremos acesso para realizar essa predição será o histórico do cliente e as informações passadas por ele no momento da reserva.

O histórico do cliente será de muita importância para o nosso desenvolvimento. Portanto, não podemos simplesmente elaborar um dataset de treino considerando todas as instâncias das tabelas.


O target será composto por um valor binário correspondendo a seguinte informação:

>**0 -** Seguiu a política do salão
>
>**1 -** Não seguiu a política do salão

Para a construção do target nós considerar a data mais recente de interação de cada cliente com o salão. Estou definindo por interação o ato do cliente faltar, cancelar ou comparecer ao serviço reservado.

Todas as demais datas anteriores a última interação serão consideradas como sendo o histórico do cliente.

Todos os clientes que faltaram ou cancelarem com uma antecedência menor do que dois dias estão infringindo a política do salão, logo, essas instâncias serão associadas ao target 1. As demais ao  target 0.

Desse modo vamos seguir da seguinte maneira:

>**1.** Obter a data mais recente de cada cliente, seja de comparecimento, falta ou cancelamento para a construção do target.
>
>**2.** Utilizar as datas passadas como histórico e obter informações relevantes.

Elaborando o target

In [12]:
cancelados_clientes = cancelados['Client'].unique().tolist()
faltantes_clientes = faltantes['Client'].unique().tolist()
transacoes_clientes = transacoes['Client'].unique().tolist()

clientes = transacoes_clientes

print(len(transacoes_clientes), len(cancelados_clientes), len(faltantes_clientes))

c = 0
for cliente_cancelado in cancelados_clientes:

    if cliente_cancelado not in clientes:
        clientes.append(cliente_cancelado)


for cliente_faltante in clientes:

    if cliente_faltante not in clientes:
        clientes.append(cliente_faltante)



767 120 45


In [13]:
dataset = pd.DataFrame({'Client':clientes})

In [14]:
dataset

,Client
0,KERT01
1,COOM01
2,PEDM01
3,BAIS01
4,FRAL01
...,...
793,CARS01
794,SHMS01
795,COLS01
796,ROUT01


In [15]:
aux = transacoes.groupby('Client')['Date'].max().to_frame('Transacao_mais_recente').reset_index()
dataset = pd.merge(dataset,aux,how='left',on='Client')

In [16]:
aux = faltantes.groupby('Client')['Date'].max().to_frame('Falta_mais_recente').reset_index()
dataset = pd.merge(dataset,aux,how='left',on='Client')

In [17]:
aux = cancelados.groupby('Client')['Booking_Date'].max().to_frame('Cancelamento_mais_recente').reset_index()
dataset = pd.merge(dataset,aux,how='left',on='Client')

In [18]:
dataset

,Client,Transacao_mais_recente,Falta_mais_recente,Cancelamento_mais_recente
0,KERT01,2018-06-20,NaT,NaT
1,COOM01,2018-06-15,NaT,NaT
2,PEDM01,2018-06-09,NaT,NaT
3,BAIS01,2018-06-09,NaT,NaT
4,FRAL01,2018-06-09,NaT,NaT
...,...,...,...,...
793,CARS01,NaT,NaT,2018-05-25
794,SHMS01,NaT,NaT,2018-07-13
795,COLS01,NaT,NaT,2018-04-22
796,ROUT01,NaT,2018-06-17,2018-05-06


In [19]:
dataset = dataset.fillna(0)
dataset['Transacao_mais_recente'] = pd.to_datetime(dataset['Transacao_mais_recente'])
dataset['Falta_mais_recente'] = pd.to_datetime(dataset['Falta_mais_recente'])
dataset['Cancelamento_mais_recente'] = pd.to_datetime(dataset['Cancelamento_mais_recente'])

Agora estamos aptos a determinar qual foi a data da interação mais recente de cada cliente com o salão

In [20]:
def interacao_mais_recente(transacao,faltante,cancelado):

    if transacao > faltante and transacao > cancelado:
        return transacao,'compareceu'
    
    elif faltante > cancelado and faltante > transacao:
        return faltante,'faltou'

    elif cancelado > faltante and cancelado > transacao:
        return cancelado,'cancelou'

In [21]:
resultado = dataset.apply(lambda x: interacao_mais_recente(x['Transacao_mais_recente'],x['Falta_mais_recente'],x['Cancelamento_mais_recente']),axis=1)

datas = []
evento = []

for i in resultado:
    datas.append(i[0])
    evento.append(i[1])

dataset['data_recente'] = datas
dataset['evento'] = evento

In [22]:
dataset = dataset.drop(['Transacao_mais_recente','Falta_mais_recente','Cancelamento_mais_recente'],axis=1)

Precisamos estabelecer o target

In [23]:
def define_target(evento, qt_dias = None ):
    if evento == 'compareceu':
        return 0
    elif evento == 'faltou':
        return 1
    
    elif evento == 'cancelou':
        if qt_dias < 2:
            return 1
        else:
            return 0

def obtem_dias(cliente,data):

    return cancelados.query(f"Booking_Date == '{data}' and Client == '{cliente}' ")['Days'].mean()

In [24]:
dataset['dias'] = dataset.apply(lambda x: obtem_dias(x['Client'],x['data_recente']) if x['evento'] == 'cancelou' else np.nan, axis=1 )

In [25]:
dataset['target'] = dataset.apply(lambda x: define_target(x['evento'],x['dias']),axis=1)

In [26]:
dataset.head()

,Client,data_recente,evento,dias,target
0,KERT01,2018-06-20,compareceu,NaN,0
1,COOM01,2018-06-15,compareceu,NaN,0
2,PEDM01,2018-06-09,compareceu,NaN,0
3,BAIS01,2018-06-09,compareceu,NaN,0
4,FRAL01,2018-06-09,compareceu,NaN,0


In [27]:
# removendo as variáveis evento e dias para evitar o target leakage
dataset = dataset.drop(['evento','dias'],axis=1)
dataset.head()

,Client,data_recente,target
0,KERT01,2018-06-20,0
1,COOM01,2018-06-15,0
2,PEDM01,2018-06-09,0
3,BAIS01,2018-06-09,0
4,FRAL01,2018-06-09,0


Agora vamos obter o histórico de cada cliente e estabelecer as demais features. O histórico consiste de datas anteriores a data mais recente que o cliente interagiu com o salão.

1. Quantidade de faltas 

In [28]:
dataset['qt_faltas'] = dataset.apply(lambda x: faltantes.query(f"Client == '{x['Client']}' and Date < '{x['data_recente']}' ").index.size,axis=1)
dataset.head()

,Client,data_recente,target,qt_faltas
0,KERT01,2018-06-20,0,0
1,COOM01,2018-06-15,0,0
2,PEDM01,2018-06-09,0,0
3,BAIS01,2018-06-09,0,0
4,FRAL01,2018-06-09,0,0


2. Staff mais frequente no histórico de faltas

In [29]:
def staff_freq_faltante(cliente, data):
    try:
        # se tiver mais de dois Staff's, não necessariamente teremos uma moda
        # Neste caso vamos considerar o primeiro nome
        return faltantes.query(f"Client == '{cliente}' and Date < '{data}'")['Staff'].mode()[0]
    except:
        return 'nenhum'
    
dataset['moda_staff_faltante'] = dataset.apply(lambda x: staff_freq_faltante(x['Client'],x['data_recente']),axis=1)

3. Serviço mais frequente no histórico de faltas

In [30]:
def servico_freq_faltante(cliente, data):
    try:
        # se tiver mais de dois serviços, não necessariamente teremos uma moda
        # Neste caso vamos considerar o primeiro nome
        return faltantes.query(f"Client == '{cliente}' and Date < '{data}'")['Service'].mode()[0]
    except:
        return 'nenhum'
    
dataset['moda_servico_faltante'] = dataset.apply(lambda x: servico_freq_faltante(x['Client'],x['data_recente']),axis=1)

4. Serviço mais frequente no histórico de cancelados

In [31]:
def servico_freq_cancelado(cliente, data):
    try:
        # se tiver mais de dois Staff's, não necessariamente teremos uma moda
        # Neste caso vamos considerar o primeiro nome
        return cancelados.query(f"Client == '{cliente}' and Booking_Date < '{data}'")['Service'].mode()[0]
    except:
        return 'nenhum'
    
dataset['moda_servico_cancelado'] = dataset.apply(lambda x: servico_freq_cancelado(x['Client'],x['data_recente']),axis=1)

5. Staff mais frequente no histórico de cancelados

In [32]:
def staff_freq_cancelado(cliente, data):
    try:
        # se tiver mais de dois Staff's, não necessariamente teremos uma moda
        # Neste caso vamos considerar o primeiro nome
        return cancelados.query(f"Client == '{cliente}' and Booking_Date < '{data}'")['Staff'].mode()[0]
    except:
        return 'nenhum'
    
dataset['moda_staff_cancelado'] = dataset.apply(lambda x: staff_freq_cancelado(x['Client'],x['data_recente']),axis=1)

6. Antecedência média, isto é, se o cliente cancelou, ele fez isso quantos dias antes da consulta ?

In [33]:
dataset['antecedencia'] = dataset.apply(lambda x: cancelados.query(f"Client == '{x['Client']}' and Booking_Date < '{x['data_recente']}' ")['Days'].mean(),axis=1)

7. Quantidade de cancelamentos presentes no histórico

In [34]:
dataset['qt_cancelamentos'] = dataset.apply(lambda x: cancelados.query(f"Client == '{x['Client']}' and Booking_Date < '{x['data_recente']}' ").index.size,axis=1)

8. Staff mais frequente no histórico de transações

In [35]:
def staff_freq_transacao(cliente, data):
    try:
        # se tiver mais de dois Staff's, não necessariamente teremos uma moda
        # Neste caso vamos considerar o primeiro nome
        return transacoes.query(f"Client == '{cliente}' and Date < '{data}'")['Staff'].mode()[0]
    except:
        return 'nenhum'
    
dataset['moda_staff_prestou_servico'] = dataset.apply(lambda x: staff_freq_transacao(x['Client'],x['data_recente']),axis=1)

9. O dia mais frequente no histórico de transações

In [36]:
def day_freq_transacao(cliente, data):
    try:
        return transacoes.query(f"Client == '{cliente}' and Date < '{data}'")['dia_nome'].mode()[0]
    except:
        return 'nenhum'
    
dataset['moda_dia'] = dataset.apply(lambda x: day_freq_transacao(x['Client'],x['data_recente']),axis=1)

10. Quantidade média histórica de serviços por dia por cliente na tabela transações

In [37]:
dataset['qte_servicos_por_dia']=dataset.apply(lambda x: transacoes.groupby(['Client','Date'])['Receipt'].size().to_frame().query(f"Client == '{x['Client']}' and Date < '{x['data_recente']}' ").mean(),axis=1)
dataset

,Client,data_recente,target,qt_faltas,moda_staff_faltante,moda_servico_faltante,moda_servico_cancelado,moda_staff_cancelado,antecedencia,qt_cancelamentos,moda_staff_prestou_servico,moda_dia,qte_servicos_por_dia
0,KERT01,2018-06-20,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,JJ,Tuesday,1.5
1,COOM01,2018-06-15,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,SINEAD,Thursday,1.0
2,PEDM01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,BECKY,Saturday,1.0
3,BAIS01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN
4,FRAL01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,CARS01,2018-05-25,1,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN
794,SHMS01,2018-07-13,1,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN
795,COLS01,2018-04-22,1,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN
796,ROUT01,2018-06-17,1,0,nenhum,nenhum,SBD,JJ,0.0,1,nenhum,nenhum,NaN


11. Estatísticas básicas acerca do pagamento de cada cliente

In [38]:
def estatisticas(dataset,transacoes):

    estats = {'Client':[],'media':[],'mediana':[],'desvio_padrao':[],'min':[],'max':[]}
    for cliente in dataset['Client'].unique().tolist():
        data_limite = dataset.query(f'Client == "{cliente}"')['data_recente'].values[0]


        dados = transacoes.query(f"Client == '{cliente}' and Date < '{data_limite}'")['Amount'].agg(['mean','median','std','min','max'])
        

        estats['Client'].append(cliente)
        estats['media'].append(dados['mean'])
        estats['mediana'].append(dados['median'])
        estats['desvio_padrao'].append(dados['std'])
        estats['min'].append(dados['min'])
        estats['max'].append(dados['max'])

    return pd.DataFrame(estats)

In [39]:
estats = estatisticas(dataset,transacoes)
estats.head()

,Client,media,mediana,desvio_padrao,min,max
0,KERT01,84.666667,82.0,16.165808,70.0,102.0
1,COOM01,70.000000,70.0,NaN,70.0,70.0
2,PEDM01,60.000000,60.0,NaN,60.0,60.0
3,BAIS01,NaN,NaN,NaN,NaN,NaN
4,FRAL01,NaN,NaN,NaN,NaN,NaN


In [40]:
dataset = pd.merge(dataset,estats,on='Client',how='inner')

12. Quantidade histórica de serviços

In [41]:
dataset['qte_servico_recebido'] = dataset.apply(lambda x: transacoes.query(f" Client == '{x['Client']}' and Date < '{x['data_recente']}'").shape[0],axis=1)

In [42]:
dataset.head(20)

,Client,data_recente,target,qt_faltas,moda_staff_faltante,moda_servico_faltante,moda_servico_cancelado,moda_staff_cancelado,antecedencia,qt_cancelamentos,moda_staff_prestou_servico,moda_dia,qte_servicos_por_dia,media,mediana,desvio_padrao,min,max,qte_servico_recebido
0,KERT01,2018-06-20,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,JJ,Tuesday,1.500000,84.666667,82.00,16.165808,70.0,102.0,3
1,COOM01,2018-06-15,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,SINEAD,Thursday,1.000000,70.000000,70.00,NaN,70.0,70.0,1
2,PEDM01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,BECKY,Saturday,1.000000,60.000000,60.00,NaN,60.0,60.0,1
3,BAIS01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0
4,FRAL01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0
5,LEVL01,2018-07-18,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,JJ,Friday,1.000000,71.500000,92.00,48.590122,0.0,102.0,4
6,JASA01,2018-07-19,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,JJ,Saturday,1.333333,111.000000,92.00,108.695293,0.0,260.0,4
7,CHOT01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0
8,KUZD01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0
9,TINT01,2018-04-05,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0


In [43]:
dataset.to_csv('../datasets_for_ml/dataset_for_train.csv',index=None)